In [1]:
import pandas as pd
import csv
pd.options.display.max_columns = 50

In [2]:
partidas = pd.read_csv("../2017/partidas.csv")

In [3]:
partidas = partidas[partidas["rodada_id"] <= 17]

In [4]:
partidas

,rodada_id,clube_casa_id,clube_visitante_id,clube_casa_posicao,clube_visitante_posicao,aproveitamento_mandante,aproveitamento_visitante,placar_oficial_mandante,placar_oficial_visitante,partida_data,local,valida
0,1,266,277,6,14,v,d,3.0,2.0,2017-05-14 11:00:00,Maracanã,True
1,1,262,282,8,10,e,e,1.0,1.0,2017-05-13 16:00:00,Maracanã,True
2,1,275,267,2,19,v,d,4.0,0.0,2017-05-14 16:00:00,Arena Palmeiras,True
3,1,264,315,9,11,e,e,1.0,1.0,2017-05-13 19:00:00,Arena Corinthians,True
4,1,283,276,7,15,v,d,1.0,0.0,2017-05-14 16:00:00,Mineirão,True
5,1,294,373,4,17,v,d,4.0,1.0,2017-05-15 20:00:00,Couto Pereira,True
6,1,284,263,5,16,v,d,2.0,0.0,2017-05-14 19:00:00,Arena do Grêmio,True
7,1,265,293,1,18,v,d,6.0,2.0,2017-05-14 16:00:00,Fonte Nova,True
8,1,303,292,3,20,v,d,4.0,0.0,2017-05-14 16:00:00,Moisés Lucarelli,True
9,1,314,287,13,12,e,e,0.0,0.0,2017-05-14 16:00:00,Ressacada,True


In [5]:
def get_pontos(row):
    home = row["placar_oficial_mandante"]
    away = row["placar_oficial_visitante"]
    if home > away:
        return 3, 0, 'v', 'd'
    elif home < away:
        return 0, 3, 'd', 'v'
    else:
        return 1, 1, 'e', 'e'

In [6]:
partidas[["pontos_mandante",
          "pontos_visitante",
          "resultado_mandante",
          "resultado_visitante"]] = partidas.apply(get_pontos, axis=1).apply(pd.Series)

In [7]:
tabela_mandante = partidas[["rodada_id",
                            "clube_casa_id",
                            "pontos_mandante",
                            "resultado_mandante",
                            "placar_oficial_mandante",
                            "placar_oficial_visitante"]]

In [8]:
tabela_visitante = partidas[["rodada_id",
                             "clube_visitante_id",
                             "pontos_visitante",
                             "resultado_visitante",
                             "placar_oficial_visitante",
                             "placar_oficial_mandante"]]

In [9]:
tabela_columns = ["rodada_id", "clube_id", "pontos", "resultado", "gols_pro", "gols_contra"]

In [10]:
tabela_mandante.columns = tabela_columns

In [11]:
tabela_visitante.columns = tabela_columns

In [12]:
tabela = pd.concat([tabela_mandante, tabela_visitante])

In [13]:
tabela = tabela.sort_values(["rodada_id", "clube_id"]).reset_index(drop=True)

In [14]:
tabela[["pontos_cumsum",
        "gols_pro_cumsum",
        "gols_contra_cumsum"]] = tabela.groupby("clube_id")[["pontos",
                                                             "gols_pro",
                                                             "gols_contra"]].cumsum()

In [15]:
tabela["aproveitamento"] = tabela.groupby("clube_id")["resultado"].apply(pd.Series.cumsum).str.slice(-5)

In [16]:
tabela["saldo_gols"] = tabela["gols_pro_cumsum"] - tabela["gols_contra_cumsum"]

In [17]:
tabela = tabela.sort_values(
    ["rodada_id", "pontos_cumsum", "saldo_gols", "gols_pro_cumsum"],
    ascending=[True, False, False, False]).reset_index(drop=True)

In [18]:
tabela["aux_posicao"] = 1

In [19]:
tabela["posicao"] = tabela.groupby("rodada_id")["aux_posicao"].cumsum()

In [20]:
partidas = pd.merge(partidas,
                    tabela,
                    left_on=["rodada_id", "clube_casa_id"],
                    right_on=["rodada_id", "clube_id"])

In [21]:
partidas = partidas.drop([
        "clube_casa_posicao",
        "clube_visitante_posicao",
        "aproveitamento_mandante",
        "aproveitamento_visitante",
        "pontos_mandante",
        "pontos_visitante",
        "resultado_mandante",
        "resultado_visitante",
        "clube_id",
        "pontos",
        "resultado",
        "gols_pro",
        "gols_contra",
        "pontos_cumsum",
        "gols_pro_cumsum",
        "gols_contra_cumsum",
        "saldo_gols",
        "aux_posicao"], axis=1)

In [22]:
partidas.columns = ['rodada_id', 'clube_casa_id', 'clube_visitante_id',
       'placar_oficial_mandante', 'placar_oficial_visitante', 'partida_data',
       'local', 'valida', 'aproveitamento_mandante', 'clube_casa_posicao']

In [23]:
partidas = pd.merge(partidas,
                    tabela,
                    left_on=["rodada_id", "clube_visitante_id"],
                    right_on=["rodada_id", "clube_id"])

In [24]:
partidas = partidas.drop([
        "clube_id",
        "pontos",
        "resultado",
        "gols_pro",
        "gols_contra",
        "pontos_cumsum",
        "gols_pro_cumsum",
        "gols_contra_cumsum",
        "saldo_gols",
        "aux_posicao"], axis=1)

In [25]:
partidas.columns = ['rodada_id', 'clube_casa_id', 'clube_visitante_id',
       'placar_oficial_mandante', 'placar_oficial_visitante', 'partida_data',
       'local', 'valida', 'aproveitamento_mandante', 'clube_casa_posicao',
       'aproveitamento_visitante', 'clube_visitante_posicao']

In [26]:
partidas = partidas[[
        'rodada_id', 'clube_casa_id', 'clube_visitante_id',
        'clube_casa_posicao', 'clube_visitante_posicao',
        'aproveitamento_mandante', 'aproveitamento_visitante',
        'placar_oficial_mandante', 'placar_oficial_visitante',
        'partida_data', 'local', 'valida']]

In [27]:
partidas

,rodada_id,clube_casa_id,clube_visitante_id,clube_casa_posicao,clube_visitante_posicao,aproveitamento_mandante,aproveitamento_visitante,placar_oficial_mandante,placar_oficial_visitante,partida_data,local,valida
0,1,266,277,6,14,v,d,3.0,2.0,2017-05-14 11:00:00,Maracanã,True
1,1,262,282,8,10,e,e,1.0,1.0,2017-05-13 16:00:00,Maracanã,True
2,1,275,267,2,19,v,d,4.0,0.0,2017-05-14 16:00:00,Arena Palmeiras,True
3,1,264,315,9,11,e,e,1.0,1.0,2017-05-13 19:00:00,Arena Corinthians,True
4,1,283,276,7,15,v,d,1.0,0.0,2017-05-14 16:00:00,Mineirão,True
5,1,294,373,4,17,v,d,4.0,1.0,2017-05-15 20:00:00,Couto Pereira,True
6,1,284,263,5,16,v,d,2.0,0.0,2017-05-14 19:00:00,Arena do Grêmio,True
7,1,265,293,1,18,v,d,6.0,2.0,2017-05-14 16:00:00,Fonte Nova,True
8,1,303,292,3,20,v,d,4.0,0.0,2017-05-14 16:00:00,Moisés Lucarelli,True
9,1,314,287,13,12,e,e,0.0,0.0,2017-05-14 16:00:00,Ressacada,True


In [28]:
partidas.to_csv("../2017/new_partidas.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, encoding="utf-8")